In [1]:
# Import relevant modules
import os
import time
from datetime import datetime
from scipy import signal

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import joblib
import pickle as pkl

In [2]:
# define parameters for retrieving files
WIN_SIZE = 32
INTENTIONS_DICT = [
    ('Mahsa', 'Obstacles15', 'T3'),
    ('Mahsa', 'Obstacles35', 'T3'),
    ('Mahsa', 'RampA', 'T3'),
    ('Mahsa', 'StraightF', 'T3'),
    ('Mahsa', 'Turn90FL', 'T3'),
    ('Mahsa', 'Turn90FR', 'T3'),
    ('Mahsa', 'Turn180L', 'T3'),
    ('Mahsa', 'Turn180R', 'T3'),
]
DICT_NUM = 4

model = 'RF'

# set the current directory
current_path = os.getcwd()

# map clusters for colours
cluster_dict = {'Cluster 1': 0, 'Cluster 2':1, 'Cluster 3':2, 'Cluster 4':3, 'Cluster 5':4, 'Cluster 6':5}

In [3]:
# columns are transitions from cluster i to cluster j, columns add up to 1
# process model parameters

def getPModel(intentionDictionary):

    prev_model = np.zeros((4,4))
    #process_model = np.ones((4,4))/4
    
    p = 0.999999
    pt = (1 - p) / 3
    process_model = np.array([[p,  pt, pt, pt],
                              [pt, p,  pt, pt],
                              [pt, pt, p,  pt],
                              [pt, pt, pt, p]])

    
    while not (prev_model == process_model).all():

        prev = 0
        
        prev_model = process_model

        p_model = np.zeros((4,4))

        for item in intentionDictionary:

            # read the data
            data = pd.read_csv(os.path.join(current_path, '2021-Results',model, '240Hz-{}-{}-{}.csv'.format(WIN_SIZE, item[1], item[0])))
            clusters = data[['Cluster 1', 'Cluster 2', 'Cluster 3', 'Cluster 4', 'Cluster 5', 'Cluster 6']]
            cluster_4 = np.array([clusters['Cluster 1'] + clusters['Cluster 4'] + clusters['Cluster 5'], clusters['Cluster 3'], 
                              clusters['Cluster 6'], clusters['Cluster 2']])

            # prior update and measurement update arrays
            p_xc_zp = np.ones(4)/4
            p_xc_zc = np.ones(4)/4

            # filtered clusters
            probBT2 = np.array([p_xc_zc])
            
            for cluster in cluster_4.T:
                p_xc_zp = process_model @ p_xc_zc
                p_xc_zc =  np.multiply(cluster, p_xc_zp) / (cluster @ p_xc_zp) 
                p_model[p_xc_zc.argmax(), prev] += 1
                prev = p_xc_zc.argmax()

        process_model = np.divide(p_model, p_model.sum(axis=0))

        print(process_model)
        
    return(process_model)

In [4]:
pModel = getPModel(INTENTIONS_DICT)

joblib.dump(pModel, 'HMP.joblib')

[[0.98692033 0.01858544 0.01616628 0.01767956]
 [0.00317083 0.97883325 0.01501155 0.01325967]
 [0.00475624 0.00103252 0.96882217 0.00110497]
 [0.0051526  0.00154879 0.         0.9679558 ]]
[[0.98553883 0.02       0.01696833 0.02169197]
 [0.00138669 0.97789474 0.02262443 0.01626898]
 [0.00653724 0.00105263 0.96040724 0.        ]
 [0.00653724 0.00105263 0.         0.96203905]]
[[0.9855331  0.02002107 0.0158371  0.02267819]
 [0.00138724 0.97787144 0.02262443 0.0161987 ]
 [0.00634166 0.00105374 0.96153846 0.        ]
 [0.00673801 0.00105374 0.         0.96112311]]
[[0.98553883 0.02002107 0.01585504 0.0227027 ]
 [0.00138669 0.97787144 0.02265006 0.01621622]
 [0.00633914 0.00105374 0.9614949  0.        ]
 [0.00673534 0.00105374 0.         0.96108108]]
[[0.98553883 0.02002107 0.01585504 0.0227027 ]
 [0.00138669 0.97787144 0.02265006 0.01621622]
 [0.00633914 0.00105374 0.9614949  0.        ]
 [0.00673534 0.00105374 0.         0.96108108]]


['HMP.joblib']

In [9]:
p = 0.25
pt = (1 - p) / 3
process_model = np.array([[p,  pt, pt, pt],
                              [pt, p,  pt, pt],
                              [pt, pt, p,  pt],
                              [pt, pt, pt, p]])

process_model[0,1]+=1

print(process_model)

[[0.25 1.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]]
